# Plot clade frequency errors by delay type and forecast horizon for natural H3N2 populations 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
sns.set_style("ticks")

## Load clade frequencies

In [ ]:
frequencies = pd.read_csv(
    "../results/clade_frequencies_for_h3n2.tsv",
    sep="\t",
    parse_dates=["timepoint", "future_timepoint"],
).rename(
    columns={"delta_month": "horizon"},
)

In [ ]:
frequencies["horizon"] = frequencies["horizon"].astype(int)

In [ ]:
frequencies.head()

In [ ]:
frequencies["frequency_error"] = frequencies["observed_frequency"] - frequencies["projected_frequency"]

In [ ]:
frequencies["abs_frequency_error"] = np.abs(frequencies["frequency_error"])

In [ ]:
frequencies.shape

## Annotate initial frequency without delay to all clades

In [ ]:
frequencies_without_delay = frequencies.loc[
    frequencies["delay_type"] == "none",
    ("clade_membership", "timepoint", "frequency")
].drop_duplicates()

In [ ]:
frequencies_without_delay.shape

In [ ]:
frequencies_without_delay = frequencies_without_delay.rename(
    columns={"frequency": "frequency_without_delay"},
)

In [ ]:
frequencies_without_delay.head()

In [ ]:
frequencies.shape

In [ ]:
frequencies = frequencies.merge(
    frequencies_without_delay,
    how="left",
    on=["timepoint", "clade_membership"],
)

In [ ]:
pd.isnull(frequencies["frequency_without_delay"]).sum()

In [ ]:
frequencies["frequency_without_delay"] = frequencies["frequency_without_delay"].fillna(0)

In [ ]:
frequencies.head()

In [ ]:
frequencies.loc[:, ["frequency_without_delay", "frequency"]]

In [ ]:
(frequencies["frequency"] > 0.1).sum()

In [ ]:
(frequencies["frequency_without_delay"] > 0.1).sum()

In [ ]:
frequencies.loc[frequencies["frequency"] > 0.1, ["clade_membership", "timepoint", "future_timepoint", "delay_type"]].drop_duplicates()

In [ ]:
frequencies.loc[frequencies["frequency_without_delay"] > 0.1, ["clade_membership", "timepoint", "future_timepoint", "delay_type"]].drop_duplicates()

In [ ]:
distinct_large_clades_with_delay = set(frequencies.loc[frequencies["frequency"] > 0.1, "clade_membership"].drop_duplicates().values)

In [ ]:
distinct_large_clades_without_delay = set(frequencies.loc[frequencies["frequency_without_delay"] > 0.1, "clade_membership"].drop_duplicates().values)

In [ ]:
len(distinct_large_clades_with_delay)

In [ ]:
len(distinct_large_clades_without_delay)

In [ ]:
distinct_large_clades_with_delay - distinct_large_clades_without_delay

In [ ]:
distinct_large_clades_without_delay - distinct_large_clades_with_delay

In [ ]:
frequencies[frequencies["clade_membership"].isin(distinct_large_clades_with_delay - distinct_large_clades_without_delay)]

## Plot clade frequency errors by delay type and forecast horizon

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=150)

sns.boxplot(
    x="horizon",
    y="frequency_error",
    hue="delay_type",
    hue_order=("none", "ideal", "observed"),
    data=frequencies,
    color="#CCCCCC",
    fliersize=0.0,
    ax=ax,
)
sns.stripplot(
    x="horizon",
    y="frequency_error",
    hue="delay_type",
    data=frequencies,
    hue_order=("none", "ideal", "observed"),
    alpha=0.35,
    ax=ax,
    dodge=True,
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    linewidth=1,
)

handles, labels = ax.get_legend_handles_labels()

ax.legend(
    handles=handles[3:],
    labels=labels[3:],
    loc="upper left",
    title="Delay type",
    frameon=False,
)
ax.set_xlabel("Forecast horizon (months)")
ax.set_ylabel("Clade frequency error\n(without delay - with delay)")

sns.despine()
plt.tight_layout()

Plot clade frequency errors for larger clades only.

In [ ]:
large_frequency_threshold = 0.1

In [ ]:
large_frequencies = frequencies.query(f"frequency_without_delay >= {large_frequency_threshold}")

In [ ]:
large_frequencies.shape

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=150)

sns.boxplot(
    x="horizon",
    y="frequency_error",
    hue="delay_type",
    hue_order=("none", "ideal", "observed"),
    data=large_frequencies,
    color="#CCCCCC",
    fliersize=0.0,
    ax=ax,
)
sns.stripplot(
    x="horizon",
    y="frequency_error",
    hue="delay_type",
    data=large_frequencies,
    hue_order=("none", "ideal", "observed"),
    alpha=0.35,
    ax=ax,
    dodge=True,
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    linewidth=1,
)

handles, labels = ax.get_legend_handles_labels()

ax.legend(
    handles=handles[3:],
    labels=labels[3:],
    loc="lower left",
    title="Delay type",
    frameon=False,
)
ax.set_xlabel("Forecast horizon (months)")
ax.set_ylabel(r"""Clade frequency error
($\geq${large_frequency_threshold}% without delay - with delay)""".format(large_frequency_threshold=int(large_frequency_threshold * 100)))

sns.despine()
plt.tight_layout()

In [ ]:
sns.lmplot(
    data=large_frequencies,
    x="frequency",
    y="frequency_error",
    hue="delay_type",
    col="horizon",
    col_wrap=2,
    height=4,
)

In [ ]:
large_frequencies

## Plot absolute clade frequency errors

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=150)

sns.boxplot(
    x="horizon",
    y="abs_frequency_error",
    hue="delay_type",
    hue_order=("none", "ideal", "observed"),
    data=large_frequencies,
    color="#CCCCCC",
    fliersize=0.0,
    ax=ax,
)
sns.stripplot(
    x="horizon",
    y="abs_frequency_error",
    hue="delay_type",
    data=large_frequencies,
    hue_order=("none", "ideal", "observed"),
    alpha=0.35,
    ax=ax,
    dodge=True,
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    linewidth=1,
)

handles, labels = ax.get_legend_handles_labels()

ax.legend(
    handles=handles[3:],
    labels=labels[3:],
    loc="upper left",
    title="Delay type",
    frameon=False,
)
ax.set_xlabel("Forecast horizon (months)")
ax.set_ylabel(r"""Absolute clade frequency error
($\geq${large_frequency_threshold}% without delay - with delay)""".format(large_frequency_threshold=int(large_frequency_threshold * 100)))

sns.despine()
plt.tight_layout()

## Plot effect of interventions on absolute clade frequency error by timepoint and clade

In [ ]:
status_quo = large_frequencies.query("(horizon == 12) & (delay_type == 'observed')").loc[
    :,
    ("future_timepoint", "clade_membership", "abs_frequency_error")
].copy()

In [ ]:
status_quo.head()

In [ ]:
improved_vaccine_dev = large_frequencies.query("(horizon == 6) & (delay_type == 'observed')").loc[
    :,
    ("future_timepoint", "clade_membership", "abs_frequency_error")
].copy()

In [ ]:
improved_vaccine_dev.head()

In [ ]:
status_quo_vs_improved_vaccine = status_quo.merge(
    improved_vaccine_dev,
    on=["future_timepoint", "clade_membership"],
    how="inner",
    suffixes=["_status_quo", "_intervention"],
)

In [ ]:
status_quo_vs_improved_vaccine["difference_in_abs_frequency_error"] = (
    status_quo_vs_improved_vaccine["abs_frequency_error_status_quo"] - status_quo_vs_improved_vaccine["abs_frequency_error_intervention"]
)

In [ ]:
status_quo_vs_improved_vaccine.head()

In [ ]:
status_quo_vs_improved_vaccine.shape

In [ ]:
improved_surveillance = large_frequencies.query("(horizon == 12) & (delay_type == 'ideal')").loc[
    :,
    ("future_timepoint", "clade_membership", "abs_frequency_error")
].copy()

In [ ]:
improved_surveillance.head()

In [ ]:
status_quo_vs_improved_surveillance = status_quo.merge(
    improved_surveillance,
    on=["future_timepoint", "clade_membership"],
    how="inner",
    suffixes=["_status_quo", "_intervention"],
)

In [ ]:
status_quo_vs_improved_surveillance.head()

In [ ]:
status_quo_vs_improved_surveillance["difference_in_abs_frequency_error"] = (
    status_quo_vs_improved_surveillance["abs_frequency_error_status_quo"] - status_quo_vs_improved_surveillance["abs_frequency_error_intervention"]
)

In [ ]:
status_quo_vs_improved_surveillance

In [ ]:
improved_vaccine_and_surveillance = large_frequencies.query("(horizon == 6) & (delay_type == 'ideal')").loc[
    :,
    ("future_timepoint", "clade_membership", "abs_frequency_error")
].copy()

In [ ]:
improved_vaccine_and_surveillance.head()

In [ ]:
status_quo_vs_improved_vaccine_and_surveillance = status_quo.merge(
    improved_vaccine_and_surveillance,
    on=["future_timepoint", "clade_membership"],
    how="inner",
    suffixes=["_status_quo", "_intervention"],
)

In [ ]:
status_quo_vs_improved_vaccine_and_surveillance.shape

In [ ]:
status_quo_vs_improved_vaccine_and_surveillance["difference_in_abs_frequency_error"] = (
    status_quo_vs_improved_vaccine_and_surveillance["abs_frequency_error_status_quo"] - status_quo_vs_improved_vaccine_and_surveillance["abs_frequency_error_intervention"]
)

In [ ]:
status_quo_vs_improved_vaccine_and_surveillance.head()

In [ ]:
interventions = pd.concat([
    pd.DataFrame({
        "intervention": "improved vaccine",
        "difference_in_abs_frequency_error": status_quo_vs_improved_vaccine["difference_in_abs_frequency_error"],
    }),
    pd.DataFrame({
        "intervention": "improved surveillance",
        "difference_in_abs_frequency_error": status_quo_vs_improved_surveillance["difference_in_abs_frequency_error"],
    }),
    pd.DataFrame({
        "intervention": "improved vaccine and surveillance",
        "difference_in_abs_frequency_error": status_quo_vs_improved_vaccine_and_surveillance["difference_in_abs_frequency_error"],
    }),
])

In [ ]:
intervention_order = [
    "improved vaccine",
    "improved surveillance",
    "improved vaccine and surveillance",
]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=150)

sns.boxplot(
    x="intervention",
    y="difference_in_abs_frequency_error",
    data=interventions,
    order=intervention_order,
    color="#CCCCCC",
    fliersize=0.0,
    ax=ax,
)
sns.stripplot(
    x="intervention",
    y="difference_in_abs_frequency_error",
    data=interventions,
    order=intervention_order,
    color="#000000",    
    alpha=0.35,
    ax=ax,
    dodge=True,
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    linewidth=1,
)

ax.set_xlabel("Intervention")
ax.set_ylabel("Difference in absolute clade frequency error\n(per clade and timepoint)")

sns.despine()
plt.tight_layout()

## Plot effect of interventions on median absolute clade frequency error (MAE) by timepoint

In [ ]:
status_quo_median_error = status_quo.groupby("future_timepoint")["abs_frequency_error"].median().reset_index()

In [ ]:
status_quo_median_error["intervention"] = "status quo"

In [ ]:
status_quo_median_error.head()

In [ ]:
improved_vaccine_dev_median_error = improved_vaccine_dev.groupby("future_timepoint")["abs_frequency_error"].median().reset_index()

In [ ]:
improved_vaccine_dev_median_error["intervention"] = "improved vaccine"

In [ ]:
improved_vaccine_dev_median_error.head()

In [ ]:
improved_surveillance_median_error = improved_surveillance.groupby("future_timepoint")["abs_frequency_error"].median().reset_index()

In [ ]:
improved_surveillance_median_error["intervention"] = "improved surveillance"

In [ ]:
improved_surveillance_median_error.head()

In [ ]:
improved_vaccine_and_surveillance_median_error = improved_vaccine_and_surveillance.groupby("future_timepoint")["abs_frequency_error"].median().reset_index()

In [ ]:
improved_vaccine_and_surveillance_median_error["intervention"] = "improved vaccine and surveillance"

In [ ]:
improved_vaccine_and_surveillance_median_error.head()

In [ ]:
median_errors = pd.concat([
    status_quo_median_error,
    improved_vaccine_dev_median_error,
    improved_surveillance_median_error,
    improved_vaccine_and_surveillance_median_error,
]).pivot_table(
    index="future_timepoint",
    columns="intervention",
    values="abs_frequency_error",
).dropna().reset_index()

In [ ]:
median_errors.head()

In [ ]:
median_errors.shape

In [ ]:
interventions_by_timepoint = pd.concat([
    pd.DataFrame({
        "intervention": "improved vaccine",
        "difference_in_mae": median_errors["status quo"] - median_errors["improved vaccine"],
    }),
    pd.DataFrame({
        "intervention": "improved surveillance",
        "difference_in_mae": median_errors["status quo"] - median_errors["improved surveillance"],
    }),
    pd.DataFrame({
        "intervention": "improved vaccine and surveillance",
        "difference_in_mae": median_errors["status quo"] - median_errors["improved vaccine and surveillance"],
    }),
])

In [ ]:
interventions_by_timepoint

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 4), dpi=150)

sns.boxplot(
    x="intervention",
    y="difference_in_mae",
    data=interventions_by_timepoint,
    order=intervention_order,
    color="#CCCCCC",
    fliersize=0.0,
    ax=ax,
)
sns.stripplot(
    x="intervention",
    y="difference_in_mae",
    data=interventions_by_timepoint,
    order=intervention_order,
    color="#000000",    
    alpha=0.35,
    ax=ax,
    dodge=True,
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    linewidth=1,
)

ax.set_xlabel("Intervention")
ax.set_ylabel("Difference in MAE\n(per timepoint)")

sns.despine()
plt.tight_layout()

In [ ]:
large_mae_frequencies = large_frequencies.groupby(["horizon", "delay_type", "timepoint"])["abs_frequency_error"].mean().reset_index()

In [ ]:
large_mae_frequencies

In [ ]:
large_mae_frequencies_by_delays = large_mae_frequencies.pivot(
    index=["horizon", "timepoint"],
    values=["abs_frequency_error"],
    columns=["delay_type"],
).fillna(0)

In [ ]:
large_mae_frequencies_by_delays.head()

In [ ]:
large_mae_frequencies_by_delays.columns = ["ideal", "none", "observed"]

In [ ]:
large_mae_frequencies_by_delays = large_mae_frequencies_by_delays.reset_index()

In [ ]:
large_mae_frequencies_by_delays

In [ ]:
large_mae_frequencies_by_delays["ideal_mae_difference"] = large_mae_frequencies_by_delays["none"] - large_mae_frequencies_by_delays["ideal"]

In [ ]:
large_mae_frequencies_by_delays["observed_mae_difference"] = large_mae_frequencies_by_delays["none"] - large_mae_frequencies_by_delays["observed"]

In [ ]:
large_mae_frequency_differences = large_mae_frequencies_by_delays.melt(
    id_vars=["horizon", "timepoint"],
    value_vars=["ideal_mae_difference", "observed_mae_difference"],
    var_name="delay_type",
    value_name="mae_difference",
)

In [ ]:
large_mae_frequency_differences["delay_type"] = large_mae_frequency_differences["delay_type"].apply(lambda delay: delay.split("_")[0])

In [ ]:
large_mae_frequency_differences

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=150)

sns.boxplot(
    x="horizon",
    y="mae_difference",
    hue="delay_type",
    hue_order=("ideal", "observed"),
    data=large_mae_frequency_differences,
    color="#CCCCCC",
    fliersize=0.0,
    ax=ax,
)
sns.stripplot(
    x="horizon",
    y="mae_difference",
    hue="delay_type",
    data=large_mae_frequency_differences,
    hue_order=("ideal", "observed"),
    alpha=0.35,
    ax=ax,
    dodge=True,
)

ax.axhline(
    y=0,
    color="#000000",
    zorder=-10,
    linewidth=1,
)

handles, labels = ax.get_legend_handles_labels()

ax.legend(
    handles=handles[2:],
    labels=labels[2:],
    loc="lower left",
    title="Delay type",
    frameon=False,
)
ax.set_xlabel("Forecast horizon (months)")
ax.set_ylabel("Difference in MAE without and with delay")

sns.despine()
plt.tight_layout()